In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

1. fetch the data from pdf

2. at lesat there should be 200 pages

In [2]:
# Load the pdf document
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(file_path="../../Class_Exercise/3-RAG_assignment/book 4 pdf.pdf")
#loader=PyPDFLoader(file_path="Arindam Choudhury 108068.pdf")
docs=loader.load()
print("Number of total pages:", len(docs))

Number of total pages: 383


3. if chunking(use the sementic chunking technique) required do chunking and then embedding

In [14]:
# embedding model for SemanticChunker and Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [4]:
# SemanticChunker and split the document
from langchain_experimental.text_splitter import SemanticChunker
text_splitter=SemanticChunker(
    embeddings=HF_EMBEDDING,
   breakpoint_threshold_type="interquartile"
)
documents=text_splitter.split_documents(docs)
print("Number of total Split documents:", len(documents))

Number of total Split documents: 815


4. store it inside the vector database(use any of them 2. astradb)

In [5]:
ASTRA_DB_API_ENDPOINT=os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKEN")
COLLECTION_NAME="astra_vector_langchain"
index_name = "vector_index"

In [6]:
# Define Astro vector store
from langchain_astradb import AstraDBVectorStore

vector_store = AstraDBVectorStore(
    collection_name=COLLECTION_NAME,
    embedding=HF_EMBEDDING,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

6. create a retriever pipeline

In [7]:
# Add documents to vector storre and create retriever

## Adding 15 documents only because of free version times out after general_method_timeout_ms = 30000 ms

vector_store.add_documents(documents=documents[0:15])
retriever = vector_store.as_retriever(
    search_kwargs={"k":3}  #get top 3 document 
)

10. then write a prompt template

In [8]:
# Custom prompt
from langchain_core.prompts import PromptTemplate

custom_prompt=PromptTemplate(
    template="""
            You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
            If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
            \nQuestion: {question}
            \nContext: {context}
            \nAnswer:
            """,
    input_variables=['context', 'questions']
)

11. generte a oputput through llm

In [9]:
# Get LLM
from langchain_google_genai import ChatGoogleGenerativeAI
LLM=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [10]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

custom_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_prompt
    | LLM
    | StrOutputParser()
)

In [12]:
custom_rag_chain.invoke("Who is the Author?")

'The provided text gives the names of several authors of different papers, including  Manigandan, K. Vijayaraja, G. Durga Revanth, A. V. S., G. Kiran Kumar, Ilaiah Kavati, Koppula Srinivas Rao, Ramalingaswamy Cheruku, Diwakar Tripathi, Y. Narasimha Reddy, Sathya Prakash Racharla, R. C. Anju, Sree Harsha Ramesh, and P.  There is no single author named.'